In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
data = np.load('board_grade.npy', allow_pickle=True)

In [4]:
x, y = data[:, 1], data[:, 0]

In [5]:
for i in range(y.shape[0]):
    try:
        assert y[i].shape == y[i+1].shape
    except:
        if i + 1 < y.shape[0]:
            print(y[i+1])

In [6]:
all_boards = np.stack(x).reshape(-1, 18, 11, 2)
all_classes = np.stack(y).reshape(-1, 7)

In [7]:
train_boards, train_classes, valid_boards, valid_classes = train_test_split(all_boards, all_classes, test_size=0.2)

In [8]:
train_boards.shape

(14264, 18, 11, 2)

In [9]:
grade_to_list = {
    "V4 (6B+)": tuple([0,0,0,0,0,0,0]),
    "V5 (6C/+)": tuple([1,0,0,0,0,0,0]),
    "V6 (7A)": tuple([1,1,0,0,0,0,0]),
    "V7 (7A+)": tuple([1,1,1,0,0,0,0]),
    "V8 (7B/+)": tuple([1,1,1,1,0,0,0]),
    "V9 (7C)": tuple([1,1,1,1,1,0,0]),
    "V10 (7C+)": tuple([1,1,1,1,1,1,0]),
    "V11 (8A) or harder": tuple([1,1,1,1,1,1,1]),
}

grade_num_to_list = {
    0: tuple([0,0,0,0,0,0,0]),
    1: tuple([1,0,0,0,0,0,0]),
    2: tuple([1,1,0,0,0,0,0]),
    3: tuple([1,1,1,0,0,0,0]),
    4: tuple([1,1,1,1,0,0,0]),
    5: tuple([1,1,1,1,1,0,0]),
    6: tuple([1,1,1,1,1,1,0]),
    7: tuple([1,1,1,1,1,1,1]),
}

print_dict = {
    0: "M",
    1: "S",
    2: "E"
}

list_to_grade = dict((v,k) for k,v in grade_to_list.items())
list_to_grade_num = dict((v,k) for k,v in grade_num_to_list.items())

def print_board_bar():
    for i in range(26):
        print("-", end="")
    print()

def print_board(board, grade_list):
    print(" "*3, end="| ")
    for i in range(11):
        print(f"{chr(i + 65)}", end=" ")
    print()
    print_board_bar()
    for i in range(18):
        print(f"{18 - i:2} |", end=" ")
        for j in range(11):
            num_printed = False
            for c in range(2):
                if board[17 - i, j, c] > 0:
                    char = "M"
                    if c > 0:
                        char = "S" if i >= 12 else "E"
                    print(char, end=" ")  
                    num_printed = True
            if not num_printed:
                print("-", end=" ")
        print()
    print_board_bar()
        
    print(f"Problem Grade: {list_to_grade[tuple([round(x) for x in grade_list])]}")

def print_problem(problem_num):
    if problem_num < 0 or problem_num > all_boards.shape[0]:
        print("problem number out of bounds!")
        return -1
    print_board(all_boards[problem_num], all_classes[problem_num])

print_problem(1)
print()
print()
print_problem(5)
print()
print()
print_problem(499)

   | A B C D E F G H I J K 
--------------------------
18 | - - - - E - - - - - - 
17 | - - - - - - - - - - - 
16 | - - - - - - - - - - - 
15 | - - - - M - - - - - - 
14 | - - - - - - - - - - - 
13 | - - - - - - M - - - - 
12 | - - - - - - - - - - - 
11 | - - - - - - - - M - M 
10 | - - - - - - - - - - - 
 9 | - - - - - - - - - - - 
 8 | - - - - - - - - - M - 
 7 | - - - - - - - - - - - 
 6 | - - - - - - - - - - - 
 5 | - - - - - S - - - - - 
 4 | - - - - - - - - - - - 
 3 | - - - - - - - - - - - 
 2 | - - - - - - - - - - - 
 1 | - - - - - - - - - - - 
--------------------------
Problem Grade: V4 (6B+)


   | A B C D E F G H I J K 
--------------------------
18 | - - E - - - - - - - - 
17 | - - - - - - - - - - - 
16 | - - - M - - - - - - - 
15 | - - - - - - - - - - - 
14 | - - - - - - - - - - - 
13 | - - - M - - - - - - - 
12 | - - - - - - - - M - - 
11 | - - - - - - - - - - - 
10 | - - - - - - - - - - - 
 9 | - - - - - M - - - - - 
 8 | - - - - - - - - - - - 
 7 | - - - - - - - - - - 

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

In [11]:
# Convert data to PyTorch tensors
x_tensor = torch.tensor(all_boards, dtype=torch.float32)
y_tensor = torch.tensor(all_classes, dtype=torch.float32)
# create the dataset
dataset = data.TensorDataset(x_tensor, y_tensor)
val_size = int(len(dataset)*0.2)
train_size = len(dataset)- int(len(dataset)*0.2)
train_dataset, val_dataset = data.random_split(dataset, [train_size, val_size])

In [12]:
INPUT_DIM = 18 * 11 * 2
OUTPUT_DIM = 7

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def calculate_accuracy(y_pred, y):
    thresh_pred = (y_pred > 0).float()
    thresh_y = (y > 0).float()
    correct = torch.tensor(0).to("cuda")
    for i in range(y.shape[0]):
        correct += thresh_y[i].equal(thresh_pred[i])
    acc = correct.float() / y.shape[0]
    return acc

def calculate_plus_minus_one(y_pred, y):
    y_pred = (y_pred > 0).float().to("cuda")
    y = (y > 0).float().to("cuda")
    correct = torch.tensor(0).to("cuda")
    for i in range(y.shape[0]):
        correct += (y_pred[i].eq(y[i]).sum() >= OUTPUT_DIM - 1)
    acc = correct.float() / y.shape[0]
    return acc

In [13]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_fc = nn.Linear(input_dim, 40)
        self.dropout = nn.Dropout(0.5)
        self.hidden_fc = nn.Linear(40, 20)
        self.output_fc = nn.Linear(20, output_dim)

    def forward(self, x):
        # x = [batch size, height, width]
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)
        # x = [batch size, height * width]
        h_1 = self.dropout(F.mish(self.input_fc(x)))
        # h_1 = [batch size, 250]
        h_2 = self.dropout(F.mish(self.hidden_fc(h_1)))
        # h_2 = [batch size, 100]
        y_pred = self.output_fc(h_2)
        # y_pred = [batch size, output dim]
        return y_pred, h_2

In [ ]:
import time
from tqdm import tqdm

# fix the random seed; don't modify this
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLP(INPUT_DIM, OUTPUT_DIM)
BATCH_SIZE = 4

train_iterator = data.DataLoader(train_dataset,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

valid_iterator = data.DataLoader(val_dataset,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([1,1,1.25,1.5,1.5,3.5,7]))
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model = model.to(device)
criterion = criterion.to(device)

def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0
    epoch_plus_minus = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        plus_minus = calculate_plus_minus_one(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_plus_minus += plus_minus.item()

    return epoch_loss / (len(iterator) * BATCH_SIZE), epoch_acc / len(iterator), epoch_plus_minus / len(iterator)

def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0
    epoch_plus_minus = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            plus_minus = calculate_plus_minus_one(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_plus_minus += plus_minus.item()

    return epoch_loss / (len(iterator) * BATCH_SIZE), epoch_acc / len(iterator), epoch_plus_minus / len(iterator)


EPOCHS = 25

best_valid_loss = float('inf')

for epoch in range(EPOCHS):

    start_time = time.monotonic()

    train_loss, train_acc, train_plus_minus = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc, valid_plus_minus = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train Acc +/- 1 Grade: {train_plus_minus*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% |  Val. Acc +/- 1 Grade: {valid_plus_minus*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 19s
	Train Loss: 0.084 | Train Acc: 41.48% | Train Acc +/- 1 Grade: 79.90%
	 Val. Loss: 0.066 |  Val. Acc: 50.20% |  Val. Acc +/- 1 Grade: 87.53%


Epoch: 02 | Epoch Time: 0m 9s
	Train Loss: 0.072 | Train Acc: 46.56% | Train Acc +/- 1 Grade: 84.34%
	 Val. Loss: 0.064 |  Val. Acc: 51.18% |  Val. Acc +/- 1 Grade: 88.03%


Epoch: 03 | Epoch Time: 0m 9s
	Train Loss: 0.070 | Train Acc: 47.11% | Train Acc +/- 1 Grade: 85.00%
	 Val. Loss: 0.064 |  Val. Acc: 51.04% |  Val. Acc +/- 1 Grade: 87.39%


Epoch: 04 | Epoch Time: 0m 9s
	Train Loss: 0.068 | Train Acc: 48.56% | Train Acc +/- 1 Grade: 85.56%
	 Val. Loss: 0.063 |  Val. Acc: 50.92% |  Val. Acc +/- 1 Grade: 87.70%


Epoch: 05 | Epoch Time: 0m 9s
	Train Loss: 0.067 | Train Acc: 48.45% | Train Acc +/- 1 Grade: 85.69%
	 Val. Loss: 0.062 |  Val. Acc: 52.41% |  Val. Acc +/- 1 Grade: 88.54%


Training:  77%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                      | 2741/3567 [00:06<00:01, 437.89it/s]

In [ ]:
import math
# grab a boulder from the validation set and see what the net thinks
view_iterator = data.DataLoader(val_dataset,
                                 shuffle=True,
                                 batch_size=1)


# set it up properly for viewing + prediction
board, grade = next(iter(view_iterator))
view_board = board.reshape(18, 11, 2)
view_grade = grade.reshape(7)
board = board.to(device)
grade = grade.to(device)
print_board(view_board, view_grade.numpy())
pred = model.eval()(board)[0].cpu()
pred_numpy = pred.detach().numpy().reshape(7)
pred_sig_numpy = torch.sigmoid(pred).detach().numpy().reshape(7)
rounded_pred = np.round(pred_sig_numpy, 1)
thresholded_pred = tuple(1 if x > 0 else 0 for x in pred_numpy)
print(f"NN Estimation: {list_to_grade[thresholded_pred]}")
print(f"Full correct class: {view_grade.numpy()}")
print(f"Model estim. class: {rounded_pred}")


In [ ]:
# create confusion matrix for model
from sklearn.metrics import f1_score, roc_auc_score

def get_stats(model, iterator, device):

    epoch_loss = 0
    epoch_acc = 0
    epoch_plus_minus = 0

    model.eval()

    pred_classes = []
    true_classes = []

    cf_pred = []
    cf_true = []

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Creating confusion matrix from model", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            # convert matrix into class
            for idx, item in enumerate(y_pred):
                max_idx_pred = max([i+1 if x > 0 else 0 for i, x in enumerate(item)])
                max_idx_true = max([i+1 if x > 0 else 0 for i, x in enumerate(y[idx])])
                pred_classes.append(tuple([1 if idx == max_idx_pred else 0 for idx in range(len(item))]))
                true_classes.append(tuple([1 if idx == max_idx_true else 0 for idx in range(len(item))]))
                cf_pred.append(max_idx_pred)
                cf_true.append(max_idx_true)

    return confusion_matrix(cf_true, cf_pred), \
        roc_auc_score(true_classes, pred_classes, average='weighted', multi_class='ovr'), \
        f1_score(true_classes, pred_classes, average='weighted')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ticklabels = ["V4", "V5", "V6", "V7", "V8", "V9", "V10", "V11+"]

# get train set stats
cf_matrix, auc, f1 = get_stats(model, train_iterator, device)
cf_matrix_norm = cf_matrix.astype('float') / cf_matrix.sum(axis=1)[:, np.newaxis]

print(f"Training set metrics:")
print(f"AUC Score: {auc:0.4f} | F1 Score: {f1:0.4f}")
ax = sns.heatmap(cf_matrix_norm, annot=True, fmt='.0%', xticklabels=ticklabels, yticklabels=ticklabels)
ax.set(xlabel="Predicted Labels", ylabel="True Labels")

In [ ]:
# get valid set stats
cf_matrix, auc, f1 = get_stats(model, valid_iterator, device)
cf_matrix_norm = cf_matrix.astype('float') / cf_matrix.sum(axis=1)[:, np.newaxis]

print(f"Validation set metrics:")
print(f"AUC Score: {auc:0.4f} | F1 Score: {f1:0.4f}")
ax = sns.heatmap(cf_matrix_norm, annot=True, fmt='.0%', xticklabels=ticklabels, yticklabels=ticklabels)
ax.set(xlabel="Predicted Labels", ylabel="True Labels")